# Optimisasi Tutor & Penjadwalan

Update:
- Sharing tutor di banyak cabang
- 2 shift sehari --> 3 shift

In [1]:
globals().clear()
import mip
import numpy
import math

## Model

In [2]:
# Himpunan Mata Pelajaran
Pelajaran = ["fis", "kim", "bio", "eko", "geo", "sej", "mat", "ind", "ing", "sos"]


######################### CONSTANTS #########################
# banyak cabang boleh lebih dari 1
# banyak kelas di setiap cabang boleh beda
# banyak kelas IPA & IPS dianggap seimbang
# 1 periode jadwal adlh 4 minggu
# 1 minggu masuk 6 hari, jd 1 periode 24 hari
# 1 hari ada 3 sesi/shift

nc = int(input("Banyak cabang (termasuk pusat): "))
np = len(Pelajaran) # banyak mapel
nh = 24 # banyak hari dalam 1 periode
ns = 3 # banyak sesi dalam sehari

nk = numpy.zeros(nc)
nipa = numpy.zeros(nc)
nips = numpy.zeros(nc)

for c in range(nc):
  nk[c] = int(input(f"Banyak kelas di cabang ke-{c+1}: "))
  nipa[c] = math.ceil(nk[c]/2) # banyak kelas IPA
  nips[c] = math.floor(nk[c]/2) # banyak kelas IPS

nk = [int(item) for item in nk]
nipa = [int(item) for item in nipa]
nips = [int(item) for item in nips]

b_1 = 2000 # gaji tutor per bulan
b_2 = 15 # biaya transport per perpindahan

In [3]:
# OBJECT model penjadwalan m
m = mip.Model("Optimimsasi Penjadwalan Bimbel")


######################### VARIABLES #########################

g = [m.add_var(var_type=mip.INTEGER) for p in range(np)] # banyak guru
t = [[m.add_var(var_type=mip.INTEGER) for h in range(nh)] for p in range(np)] # banyaknya transfer

x = []
y = []

for c in range(nc):

  # slot jadwal
  x.append([[[[m.add_var(var_type=mip.BINARY) for s in range(ns)] for h in range(nh)] for p in range(np)] for k in range(nk[c])])

  # variabel yg memutuskan bahwa kelompok k masuk hari h
  y.append([[m.add_var(var_type=mip.BINARY) for h in range(nh)] for k in range(nk[c])])


######################### OBJECTIVE FUNCTION #########################

m.objective = mip.minimize(b_1*mip.xsum(g[p] for p in range(np)) + b_2*mip.xsum(t[p][h] for p in range(np) for h in range(nh)))

In [4]:
######################### CONSTRAINTS #########################

# kendala 1 : setiap kelompok mendapat 4 sesi per minggu
for c in range(nc):
  for k in range(nk[c]):
    m += mip.xsum(x[c][k][p][h][s] for s in range(ns) for h in range(6) for p in range(np)) == 4
    m += mip.xsum(x[c][k][p][h][s] for s in range(ns) for h in range(6,12) for p in range(np)) == 4
    m += mip.xsum(x[c][k][p][h][s] for s in range(ns) for h in range(12,18) for p in range(np)) == 4
    m += mip.xsum(x[c][k][p][h][s] for s in range(ns) for h in range(18,24) for p in range(np)) == 4

# kendala 2.1 : setiap kelompok masuk pada hari yang sama setiap minggu
for c in range(nc):
  for k in range(nk[c]):
    for h in range(6):
      m += y[c][k][h+6] == y[c][k][h]
      m += y[c][k][h+12] == y[c][k][h]
      m += y[c][k][h+18] == y[c][k][h]

# kendala 3 : setiap kelompok tidak boleh masuk 2 hari berturut-turut
for c in range(nc):
  for k in range(nk[c]):
    for h in range(5):
        m += y[c][k][h] + y[c][k][h+1] <= 1

# kendala 4 : setiap kelompok masuk 2 sesi pada hari belajar mereka

for c in range(nc):
  for k in range(nk[c]):
    for h in range(nh):
      m += mip.xsum(x[c][k][p][h][s] for s in range(ns) for p in range(np)) == 2*y[c][k][h]
      m += mip.xsum(x[c][k][p][h][0] + x[c][k][p][h][ns-1] for p in range(np)) <= 1

# kendala 5 : setiap sesi hanya belajar 1 pelajaran
for c in range(nc):
  for k in range(nk[c]):
    for h in range(nh):
      for s in range(ns):
        m += mip.xsum(x[c][k][p][h][s] for p in range(np)) <= 1

# kendala 6 : kendala beban pelajaran untuk satu periode
for c in range(nc):
  for k in range(nk[c]):
    m += mip.xsum(x[c][k][6][h][s] for s in range(ns) for h in range(nh)) == 3 # mat
    m += mip.xsum(x[c][k][7][h][s] for s in range(ns) for h in range(nh)) == 2 # ind
    m += mip.xsum(x[c][k][8][h][s] for s in range(ns) for h in range(nh)) == 2 # ing

for c in range(nc):
  for k in range(nipa[c]):
    m += mip.xsum(x[c][k][0][h][s] for s in range(ns) for h in range(nh)) == 3 # fis
    m += mip.xsum(x[c][k][1][h][s] for s in range(ns) for h in range(nh)) == 3 # kim
    m += mip.xsum(x[c][k][2][h][s] for s in range(ns) for h in range(nh)) == 3 # bio

for c in range(nc):
  for k in range(nips[c]):
    m += mip.xsum(x[c][k+nipa[c]][3][h][s] for s in range(ns) for h in range(nh)) == 3 # eko
    m += mip.xsum(x[c][k+nipa[c]][4][h][s] for s in range(ns) for h in range(nh)) == 2 # geo
    m += mip.xsum(x[c][k+nipa[c]][5][h][s] for s in range(ns) for h in range(nh)) == 1 # sej
    m += mip.xsum(x[c][k+nipa[c]][9][h][s] for s in range(ns) for h in range(nh)) == 3 # sos

# kendala 7 : setiap pelajaran tidak boleh berulang dalam minggu yang sama
for c in range(nc):
  for k in range(nk[c]):
    for p in range(np):
      m += mip.xsum(x[c][k][p][h][s] for s in range(ns) for h in range(0,6)) <= 1
      m += mip.xsum(x[c][k][p][h][s] for s in range(ns) for h in range(6,12)) <= 1
      m += mip.xsum(x[c][k][p][h][s] for s in range(ns) for h in range(12,18)) <= 1
      m += mip.xsum(x[c][k][p][h][s] for s in range(ns) for h in range(18,24)) <= 1

# kendala 8 : banyak guru
for p in range(np):
  m += g[p] >= 1

# kendala 9 : setiap hari, mapel yang sama tidak melebihi gurunya
# mapel yg sama = total mapel tsb di semua cabang
# kendala ini memungkinkan untuk sharing guru
for p in range(np):
  for h in range(nh):
    for s in range(ns):
      m += mip.xsum(x[c][k][p][h][s] for c in range(nc) for k in range(nk[c])) - g[p] <= 0

# kendala 2.2 : setiap kelompok masuk pada sesi yang sama
for c in range(nc):
  for k in range(nk[c]):
    for h in range(6):
      for s in range(ns):
        m += mip.xsum(x[c][k][p][h+6][s] for p in range(np)) - mip.xsum(x[c][k][p][h][s] for p in range(np)) == 0
        m += mip.xsum(x[c][k][p][h+12][s] for p in range(np)) - mip.xsum(x[c][k][p][h][s] for p in range(np)) == 0
        m += mip.xsum(x[c][k][p][h+18][s] for p in range(np)) - mip.xsum(x[c][k][p][h][s] for p in range(np)) == 0

# kendala 10: guru tidak bisa transfer pada 2 sesi berurutan
for p in range(np):
  for h in range(nh):
    for c in range(nc):
      for s in range(ns-1):
        m += ( mip.xsum(x[c][k][p][h][s] for k in range(nk[c])) 
              + mip.xsum(x[j][k][p][h][s+1] for j in range(nc) for k in range(nk[j]))
              - mip.xsum(x[c][k][p][h][s+1] for k in range(nk[c])) - g[p] <= 0 )
        
# kendala 11: banyak perpindahan/transfer
for p in range(np):
  for h in range(nh):
    for c in range(nc):
      m += (mip.xsum(x[c][k][p][h][0] for k in range(nk[c])) 
            + mip.xsum(x[j][k][p][h][2] for j in range(nc) for k in range(nk[j])) 
            - mip.xsum(x[c][k][p][h][2] for k in range(nk[c])) - g[p] - t[p][h] <= 0)

######################### OPTIMIZATION & SOLUTION #########################
m.optimize()
print(m.status) # status optimisasi (feasibel/tidak)
print(m.objective_value) # nilai f. objektif (total biaya gaji & transport)

OptimizationStatus.OPTIMAL
20000.0


## Hasil

In [5]:
# Hasil 1: Banyak tutor minimum untuk setiap mapel
for p in range(np):
  print("Guru", Pelajaran[p], "=", g[p].x) # note: var.x = nilai solusi dari var

Guru fis = 1.0
Guru kim = 1.0
Guru bio = 1.0
Guru eko = 1.0
Guru geo = 1.0
Guru sej = 1.0
Guru mat = 1.0
Guru ind = 1.0
Guru ing = 1.0
Guru sos = 1.0


In [6]:
def cetak_jadwal(c):
  print("Cabang", c)
  H = "Hari"
  # note: range i = banyak minggu
  for i in range(4):
    print("Minggu", i+1)
    print("Hari", end=',')
    for k in range(nk[c]):
      print("Kel.", k+1, end=',')
    print()
    for h in range(i*6,i*6+6):
      for s in range(ns):
        if (h+1)%6 == 1: H = "Senin"
        elif (h+1)%6 == 2: H = "Selasa"
        elif (h+1)%6 == 3: H = "Rabu"
        elif (h+1)%6 == 4: H = "Kamis"
        elif (h+1)%6 == 5: H = "Jumat"
        elif (h+1)%6 == 0: H = "Sabtu"
        print(H, end=',')
        for k in range(nk[c]):
          for p in range(np):
            print(Pelajaran[p] + ' ' if x[c][k][p][h][s].x>0.5 else "", end='')
          print('', end=',')
        print()

In [7]:
# Hasil 2: Jadwal
# Jalankan dan copy ke excel (atau ke spreadsheet lalu split text to columns)

cetak_jadwal(1) # Jadwal untuk cabang ke-c. Dihitung mulai dari 0

Cabang 1
Minggu 1
Hari,Kel. 1,Kel. 2,Kel. 3,Kel. 4,Kel. 5,Kel. 6,
Senin,,,fis ,,,,
Senin,bio ,,ing ,,,,
Senin,fis ,,,,,,
Selasa,,,,mat ,sos ,,
Selasa,,ind ,,eko ,mat ,sos ,
Selasa,,bio ,,,,mat ,
Rabu,,,,,,,
Rabu,,,,,,,
Rabu,,,,,,,
Kamis,,,mat ,,,,
Kamis,kim ,,ind ,,,,
Kamis,mat ,,,,,,
Jumat,,,,,,,
Jumat,,mat ,,,,,
Jumat,,fis ,,,,,
Sabtu,,,,ind ,,ing ,
Sabtu,,,,sos ,ing ,eko ,
Sabtu,,,,,ind ,,
Minggu 2
Hari,Kel. 1,Kel. 2,Kel. 3,Kel. 4,Kel. 5,Kel. 6,
Senin,,,fis ,,,,
Senin,ing ,,ind ,,,,
Senin,fis ,,,,,,
Selasa,,,,ind ,sej ,,
Selasa,,bio ,,eko ,mat ,ing ,
Selasa,,ing ,,,,sos ,
Rabu,,,,,,,
Rabu,,,,,,,
Rabu,,,,,,,
Kamis,,,kim ,,,,
Kamis,kim ,,bio ,,,,
Kamis,bio ,,,,,,
Jumat,,,,,,,
Jumat,,kim ,,,,,
Jumat,,ind ,,,,,
Sabtu,,,,ing ,,eko ,
Sabtu,,,,geo ,eko ,ind ,
Sabtu,,,,,geo ,,
Minggu 3
Hari,Kel. 1,Kel. 2,Kel. 3,Kel. 4,Kel. 5,Kel. 6,
Senin,,,kim ,,,,
Senin,kim ,,bio ,,,,
Senin,bio ,,,,,,
Selasa,,,,mat ,eko ,,
Selasa,,mat ,,sej ,sos ,eko ,
Selasa,,fis ,,,,sos ,
Rabu,,,,,,,
Rabu,,,,,,,
Rabu,,,